# Pytest Fixtures

## What You'll Learn
- What are fixtures and why use them?
- Creating basic fixtures
- Using fixtures in tests
- Fixture scopes (function, class, module, session)
- Setup and teardown with fixtures
- Fixture best practices

---

## The Problem: Repetitive Setup Code

Without fixtures, you end up repeating setup code in every test.

### Example: Testing a Database Class

In [ ]:
# database.py
class Database:
    """Simple database class."""
    
    def __init__(self):
        self.data = {}
    
    def add(self, key, value):
        """Add item to database."""
        self.data[key] = value
    
    def get(self, key):
        """Get item from database."""
        return self.data.get(key)
    
    def delete(self, key):
        """Delete item from database."""
        if key in self.data:
            del self.data[key]
    
    def count(self):
        """Return number of items."""
        return len(self.data)

### Without Fixtures (Repetitive Code)

In [ ]:
# test_database_without_fixtures.py
from database import Database

def test_add_item():
    db = Database()  # Create database
    db.add("name", "Alice")
    assert db.get("name") == "Alice"

def test_get_item():
    db = Database()  # Create database again
    db.add("age", 25)
    assert db.get("age") == 25

def test_delete_item():
    db = Database()  # Create database again
    db.add("city", "NYC")
    db.delete("city")
    assert db.get("city") is None

def test_count():
    db = Database()  # Create database again
    db.add("x", 1)
    db.add("y", 2)
    assert db.count() == 2

**Problems:**
- ❌ Repeating `db = Database()` in every test
- ❌ Hard to maintain - if setup changes, update everywhere
- ❌ Verbose and cluttered

**Solution:** Fixtures! 🎉

---

## What Are Fixtures?

**Fixtures** are functions that provide test data or setup/teardown code. They:
- ✅ Set up test data before tests run
- ✅ Can be reused across multiple tests
- ✅ Clean up after tests (if needed)
- ✅ Make tests cleaner and more maintainable

Think of fixtures as **"test ingredients"** - you prepare them once and use them in multiple tests.

---

## Creating Your First Fixture

### Step 1: Define a Fixture

In [ ]:
import pytest
from database import Database

@pytest.fixture
def db():
    """Create a database for testing."""
    database = Database()
    return database

**What happened:**
- `@pytest.fixture` decorator makes this a fixture
- Function name (`db`) is how you'll use it in tests
- Returns the object you want to use in tests

### Step 2: Use the Fixture in Tests

In [ ]:
def test_add_item(db):  # ← Notice the parameter name matches fixture name
    """Test adding an item."""
    db.add("name", "Alice")
    assert db.get("name") == "Alice"

def test_get_item(db):
    """Test getting an item."""
    db.add("age", 25)
    assert db.get("age") == 25

def test_delete_item(db):
    """Test deleting an item."""
    db.add("city", "NYC")
    db.delete("city")
    assert db.get("city") is None

def test_count(db):
    """Test counting items."""
    db.add("x", 1)
    db.add("y", 2)
    assert db.count() == 2

**Magic! ✨**
- Just add `db` as a parameter to your test
- Pytest automatically calls the `db()` fixture and passes the result
- Each test gets a fresh database
- No more repetition!

---

## Fixture with Initial Data

Fixtures can do more than just create objects - they can populate them with test data.

In [ ]:
@pytest.fixture
def db_with_users():
    """Create a database with sample users."""
    database = Database()
    database.add("user1", {"name": "Alice", "age": 25})
    database.add("user2", {"name": "Bob", "age": 30})
    database.add("user3", {"name": "Charlie", "age": 35})
    return database

def test_get_user(db_with_users):
    """Test retrieving a user."""
    user = db_with_users.get("user1")
    assert user["name"] == "Alice"
    assert user["age"] == 25

def test_count_users(db_with_users):
    """Test counting users."""
    assert db_with_users.count() == 3

def test_delete_user(db_with_users):
    """Test deleting a user."""
    db_with_users.delete("user2")
    assert db_with_users.count() == 2
    assert db_with_users.get("user2") is None

---

## Multiple Fixtures in One Test

You can use multiple fixtures in a single test!

In [ ]:
@pytest.fixture
def sample_user():
    """Create a sample user."""
    return {"name": "Alice", "email": "alice@example.com", "age": 25}

@pytest.fixture
def empty_db():
    """Create an empty database."""
    return Database()

def test_add_user_to_db(empty_db, sample_user):
    """Test adding a user to the database."""
    empty_db.add("user1", sample_user)
    
    retrieved_user = empty_db.get("user1")
    assert retrieved_user["name"] == "Alice"
    assert retrieved_user["email"] == "alice@example.com"

---

## Fixture Scopes

By default, fixtures are created fresh for **each test function**. But sometimes you want to reuse the same fixture across multiple tests.

### Available Scopes:

1. **`function`** (default) - Created for each test function
2. **`class`** - Created once per test class
3. **`module`** - Created once per test file
4. **`session`** - Created once per entire test session

### Scope: Function (Default)

In [ ]:
@pytest.fixture  # Same as @pytest.fixture(scope="function")
def counter():
    """Create a new counter for each test."""
    print("\n🔵 Creating counter")
    return {"count": 0}

def test_increment_1(counter):
    counter["count"] += 1
    print(f"Test 1: count = {counter['count']}")
    assert counter["count"] == 1

def test_increment_2(counter):
    counter["count"] += 1
    print(f"Test 2: count = {counter['count']}")
    assert counter["count"] == 1  # Still 1, new counter!

**Run with:** `pytest -s` (to see print statements)

**Output:**
```
🔵 Creating counter
Test 1: count = 1
.
🔵 Creating counter
Test 2: count = 1
.
```

Each test gets a fresh counter!

### Scope: Module

When you want to create a fixture **once per file** and share it across all tests.

In [ ]:
@pytest.fixture(scope="module")
def expensive_setup():
    """
    Simulate expensive setup (database connection, loading large file, etc.)
    Created once per test module.
    """
    print("\n🟢 Setting up expensive resource (only once!)")
    # Imagine this takes 5 seconds
    return {"data": "expensive data"}

def test_use_setup_1(expensive_setup):
    print(f"Test 1 using: {expensive_setup['data']}")
    assert expensive_setup["data"] == "expensive data"

def test_use_setup_2(expensive_setup):
    print(f"Test 2 using: {expensive_setup['data']}")
    assert expensive_setup["data"] == "expensive data"

def test_use_setup_3(expensive_setup):
    print(f"Test 3 using: {expensive_setup['data']}")
    assert expensive_setup["data"] == "expensive data"

**Output:**
```
🟢 Setting up expensive resource (only once!)
Test 1 using: expensive data
.
Test 2 using: expensive data
.
Test 3 using: expensive data
.
```

Setup happens only once! All tests share the same fixture.

**⚠️ Warning:** Be careful! Changes in one test can affect others.

### Scope Comparison Example

In [ ]:
@pytest.fixture(scope="function")
def function_fixture():
    print("\n  ⚡ Function-scoped fixture")
    return "function"

@pytest.fixture(scope="module")
def module_fixture():
    print("\n🏢 Module-scoped fixture")
    return "module"

def test_a(function_fixture, module_fixture):
    print(f"    Test A: {function_fixture}, {module_fixture}")

def test_b(function_fixture, module_fixture):
    print(f"    Test B: {function_fixture}, {module_fixture}")

def test_c(function_fixture, module_fixture):
    print(f"    Test C: {function_fixture}, {module_fixture}")

**Output:**
```
🏢 Module-scoped fixture          ← Created once
  ⚡ Function-scoped fixture       ← Created for test A
    Test A: function, module
  ⚡ Function-scoped fixture       ← Created for test B
    Test B: function, module
  ⚡ Function-scoped fixture       ← Created for test C
    Test C: function, module
```

---

## Setup and Teardown with Fixtures

Fixtures can also **clean up** after tests using the `yield` statement.

### Basic Teardown Pattern

In [ ]:
@pytest.fixture
def database_connection():
    """Simulate database connection with setup and teardown."""
    print("\n🔌 Connecting to database...")
    connection = {"connected": True}
    
    yield connection  # Provide to test
    
    # Everything after yield runs AFTER the test
    print("\n🔌 Closing database connection...")
    connection["connected"] = False

def test_query_database(database_connection):
    print("  📊 Running query...")
    assert database_connection["connected"] is True

**Output:**
```
🔌 Connecting to database...
  📊 Running query...
🔌 Closing database connection...
```

**How it works:**
1. Code before `yield` → Setup (runs before test)
2. `yield connection` → Provides fixture to test
3. Test runs
4. Code after `yield` → Teardown (runs after test)

### Real-World Example: File Handling

In [ ]:
import os

@pytest.fixture
def temp_file():
    """Create a temporary file and clean it up after test."""
    filename = "test_data.txt"
    
    # Setup: Create file
    print(f"\n📝 Creating {filename}")
    with open(filename, "w") as f:
        f.write("test data")
    
    yield filename  # Provide filename to test
    
    # Teardown: Delete file
    print(f"\n🗑️ Deleting {filename}")
    if os.path.exists(filename):
        os.remove(filename)

def test_read_file(temp_file):
    """Test reading from file."""
    print(f"  📖 Reading {temp_file}")
    with open(temp_file, "r") as f:
        content = f.read()
    assert content == "test data"

def test_file_exists(temp_file):
    """Test file exists."""
    print(f"  🔍 Checking if {temp_file} exists")
    assert os.path.exists(temp_file)

**Each test:**
1. Creates the file (setup)
2. Runs the test
3. Deletes the file (teardown)

No leftover test files! 🎉

---

## Fixture Composition (Fixtures Using Fixtures)

Fixtures can use other fixtures!

In [ ]:
@pytest.fixture
def user_data():
    """Basic user data."""
    return {"name": "Alice", "email": "alice@example.com"}

@pytest.fixture
def user_with_age(user_data):
    """User data with age added."""
    user_data["age"] = 25
    return user_data

@pytest.fixture
def user_with_address(user_with_age):
    """User data with age and address."""
    user_with_age["address"] = "123 Main St"
    return user_with_age

def test_complete_user(user_with_address):
    """Test user has all data."""
    assert user_with_address["name"] == "Alice"
    assert user_with_address["email"] == "alice@example.com"
    assert user_with_address["age"] == 25
    assert user_with_address["address"] == "123 Main St"

**Chain of fixtures:**
1. `user_data` creates basic user
2. `user_with_age` uses `user_data` and adds age
3. `user_with_address` uses `user_with_age` and adds address
4. Test uses `user_with_address` and gets everything!

---

## Practical Examples

### Example 1: Shopping Cart Tests

In [ ]:
# shopping_cart.py
class ShoppingCart:
    def __init__(self):
        self.items = []
    
    def add_item(self, item, price):
        self.items.append({"item": item, "price": price})
    
    def total(self):
        return sum(item["price"] for item in self.items)
    
    def item_count(self):
        return len(self.items)
    
    def clear(self):
        self.items = []

In [ ]:
# test_shopping_cart.py
from shopping_cart import ShoppingCart

@pytest.fixture
def empty_cart():
    """Create an empty shopping cart."""
    return ShoppingCart()

@pytest.fixture
def cart_with_items():
    """Create a cart with sample items."""
    cart = ShoppingCart()
    cart.add_item("Book", 15.99)
    cart.add_item("Pen", 2.50)
    cart.add_item("Notebook", 5.99)
    return cart

def test_empty_cart_total(empty_cart):
    """Test empty cart has zero total."""
    assert empty_cart.total() == 0
    assert empty_cart.item_count() == 0

def test_add_item(empty_cart):
    """Test adding item to cart."""
    empty_cart.add_item("Book", 15.99)
    assert empty_cart.item_count() == 1
    assert empty_cart.total() == 15.99

def test_cart_total(cart_with_items):
    """Test calculating total."""
    expected = 15.99 + 2.50 + 5.99
    assert cart_with_items.total() == expected

def test_cart_item_count(cart_with_items):
    """Test item count."""
    assert cart_with_items.item_count() == 3

def test_clear_cart(cart_with_items):
    """Test clearing cart."""
    cart_with_items.clear()
    assert cart_with_items.item_count() == 0
    assert cart_with_items.total() == 0

### Example 2: User Authentication

In [ ]:
# auth.py
class AuthSystem:
    def __init__(self):
        self.users = {}
        self.logged_in_users = set()
    
    def register(self, username, password):
        if username in self.users:
            return False
        self.users[username] = password
        return True
    
    def login(self, username, password):
        if username in self.users and self.users[username] == password:
            self.logged_in_users.add(username)
            return True
        return False
    
    def logout(self, username):
        self.logged_in_users.discard(username)
    
    def is_logged_in(self, username):
        return username in self.logged_in_users

In [ ]:
# test_auth.py
from auth import AuthSystem

@pytest.fixture
def auth_system():
    """Create a fresh auth system."""
    return AuthSystem()

@pytest.fixture
def auth_with_user(auth_system):
    """Create auth system with a registered user."""
    auth_system.register("alice", "password123")
    return auth_system

def test_register_new_user(auth_system):
    """Test registering a new user."""
    result = auth_system.register("bob", "secret")
    assert result is True
    assert "bob" in auth_system.users

def test_register_duplicate_user(auth_with_user):
    """Test registering duplicate username fails."""
    result = auth_with_user.register("alice", "different_password")
    assert result is False

def test_login_success(auth_with_user):
    """Test successful login."""
    result = auth_with_user.login("alice", "password123")
    assert result is True
    assert auth_with_user.is_logged_in("alice")

def test_login_wrong_password(auth_with_user):
    """Test login with wrong password."""
    result = auth_with_user.login("alice", "wrongpassword")
    assert result is False
    assert not auth_with_user.is_logged_in("alice")

def test_logout(auth_with_user):
    """Test logout."""
    auth_with_user.login("alice", "password123")
    assert auth_with_user.is_logged_in("alice")
    
    auth_with_user.logout("alice")
    assert not auth_with_user.is_logged_in("alice")

---

## Best Practices

### 1. Fixture Naming

**✅ Good - Descriptive names:**
```python
@pytest.fixture
def empty_database():
    ...

@pytest.fixture
def user_with_admin_role():
    ...

@pytest.fixture
def temp_directory():
    ...
```

**❌ Bad - Vague names:**
```python
@pytest.fixture
def data():
    ...

@pytest.fixture
def obj():
    ...
```

### 2. Keep Fixtures Simple

**✅ Good - Focused fixture:**
```python
@pytest.fixture
def user():
    return {"name": "Alice", "age": 25}
```

**❌ Bad - Too much logic:**
```python
@pytest.fixture
def complex_setup():
    # 50 lines of setup code
    # Reading files
    # Making API calls
    # Complex calculations
    ...
```

### 3. Use Appropriate Scope

**Use `function` scope (default) when:**
- Tests need isolated, fresh data
- Fixture is cheap to create
- Tests might modify the fixture

**Use `module` scope when:**
- Fixture is expensive to create (database connection, loading large file)
- Tests only read from the fixture (don't modify)
- All tests can safely share the same instance

### 4. Clean Up Resources

**Always clean up in fixtures that create resources:**

In [ ]:
@pytest.fixture
def resource():
    # Create resource
    resource = create_resource()
    
    yield resource
    
    # Always clean up
    resource.close()
    cleanup()

---

## Common Fixture Patterns

### Pattern 1: Database Fixture

In [ ]:
@pytest.fixture(scope="module")
def db_connection():
    """Create database connection once per module."""
    print("Connecting to database...")
    connection = connect_to_db()
    
    yield connection
    
    print("Closing database connection...")
    connection.close()

@pytest.fixture
def clean_db(db_connection):
    """Provide clean database for each test."""
    yield db_connection
    # Clean up after each test
    db_connection.clear_all_data()

### Pattern 2: Sample Data Fixture

In [ ]:
@pytest.fixture
def sample_users():
    """Provide list of sample users."""
    return [
        {"name": "Alice", "age": 25, "role": "admin"},
        {"name": "Bob", "age": 30, "role": "user"},
        {"name": "Charlie", "age": 35, "role": "user"}
    ]

def test_filter_admins(sample_users):
    admins = [u for u in sample_users if u["role"] == "admin"]
    assert len(admins) == 1
    assert admins[0]["name"] == "Alice"

### Pattern 3: Configuration Fixture

In [ ]:
@pytest.fixture
def app_config():
    """Provide test configuration."""
    return {
        "debug": True,
        "testing": True,
        "database": "test_db",
        "api_key": "test_key_12345"
    }

def test_config_has_debug(app_config):
    assert app_config["debug"] is True

def test_config_has_test_database(app_config):
    assert app_config["database"] == "test_db"

---

## Summary

### What We Learned:

✅ **Fixtures** - Reusable setup code for tests  
✅ **@pytest.fixture** - Decorator to create fixtures  
✅ **Using fixtures** - Add as test function parameters  
✅ **Scopes** - Control fixture lifespan (function, module, class, session)  
✅ **Setup/Teardown** - Use `yield` for cleanup  
✅ **Composition** - Fixtures can use other fixtures  

### Key Patterns:

```python
# Basic fixture
@pytest.fixture
def my_fixture():
    return some_value

# With teardown
@pytest.fixture
def resource():
    r = create()
    yield r
    r.cleanup()

# With scope
@pytest.fixture(scope="module")
def expensive_resource():
    return setup_expensive_thing()
```

### Next Steps:

Next, we'll learn about **parametrization** - how to run the same test with different inputs!